In [1]:
import numpy as np # фреймы данных
import pandas as pd # графики
import matplotlib as mpl
# стили и шаблоны графиков на основе matplotlib
import seaborn as sns
# для таймера
import time
# загрузка пакетов: модели ------------------------------------------------- # SVM
from sklearn.svm import SVC
# логистическая рагрессия
from sklearn.linear_model import LogisticRegression # стандартизация
from sklearn.preprocessing import StandardScaler
# метод главных компонент
from sklearn.decomposition import PCA
# конвейеры
from sklearn.pipeline import make_pipeline
# перекрёстная проверка и метод проверочной выборки
from sklearn.model_selection import cross_val_score, train_test_split # для перекрёстной проверки и сеточного поиска
from sklearn.model_selection import KFold, GridSearchCV
# сводка по точности классификации
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

# константы
# ядро для генератора случайных чисел
my_seed = 11
# создаём псевдоним для короткого обращения к графикам
plt = mpl.pyplot
# настройка стиля и отображения графиков
# примеры стилей и шаблонов графиков:
# http://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html mpl.style.use('seaborn-whitegrid')
sns.set_palette("Set2")
# раскомментируйте следующую строку, чтобы посмотреть палитру
# sns.color_palette("Set2")

In [2]:
fileURL = 'https://raw.githubusercontent.com/aksyuk/MTML/main/Labs/data/winequality-white_for_lab.csv'

DF_raw = pd.read_csv(fileURL)

DF_raw = DF_raw[['fixed_acidity','volatile_acidity','citric_acid','residual_sugar','chlorides','free_sulfur_dioxide','pH','sulphates','alcohol','Y']]
    

In [3]:
# наблюдения для моделирования
DF = DF_raw.sample(frac = 0.85, random_state = my_seed) # отложенные наблюдения
DF_predict = DF_raw.drop(DF.index)
     

In [4]:
DF_raw.dtypes

fixed_acidity          float64
volatile_acidity       float64
citric_acid            float64
residual_sugar         float64
chlorides              float64
free_sulfur_dioxide    float64
pH                     float64
sulphates              float64
alcohol                float64
Y                        int64
dtype: object

Все переменные имеют числовой тип

In [5]:
# считаем пропуски в каждом столбце
DF_raw.isna().sum()

fixed_acidity          0
volatile_acidity       0
citric_acid            0
residual_sugar         0
chlorides              0
free_sulfur_dioxide    0
pH                     0
sulphates              0
alcohol                0
Y                      0
dtype: int64

Пропусков в таблице нет

## Стандартизация и переход к главным компонентам

In [6]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# стандартизация
sc = StandardScaler()
X_train_std = sc.fit_transform(DF.iloc[:, 2:].values)
# оцениваем объяснённую главными компонентами дисперсию
pca = PCA()
X_train_pca = pca.fit_transform(X_train_std)
# считаем доли объяснённой дисперсии
frac_var_expl = pca.explained_variance_ratio_
print('Доли объяснённой дисперсии по компонентам в PLS:\n',
 np.around(frac_var_expl, 3),
 '\nОбщая сумма долей:', np.around(sum(frac_var_expl), 3))

Доли объяснённой дисперсии по компонентам в PLS:
 [0.252 0.149 0.142 0.129 0.103 0.093 0.081 0.051] 
Общая сумма долей: 1.0


Таким образом, первые две главные компоненты объясняют 38.3% разброса 9 объясняющих переменных.

In [7]:
X_train = DF[['fixed_acidity','volatile_acidity','citric_acid','residual_sugar','chlorides','free_sulfur_dioxide','pH','sulphates','alcohol']]
y_train = DF.Y

### Модель №1

In [8]:
pipe_svc = make_pipeline(StandardScaler(), SVC(random_state=my_seed))
# настроим параметры SVM с помощью сеточного поиска

param_grid = [{'svc__C': [0.1, 100, 80],
              'svc__gamma': [1, 0.1, 0.2, 0.001],
              'svc__kernel': ['rbf']}]

# разбиения для перекрёстной проверки
kfold = KFold(n_splits=5, random_state=my_seed, shuffle=True)
gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, scoring='accuracy', refit=True, cv=kfold,
n_jobs=-1)
# таймер
tic = time.perf_counter()
# запускаем сеточный поиск
gs = gs.fit(X_train, y_train)
# таймер
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

Сеточный поиск занял 13.34 секунд


In [9]:
# точность лучшей модели
np.around(gs.best_score_, 3)

0.798

In [10]:
gs.best_estimator_.get_params()

{'memory': None,
 'steps': [('standardscaler', StandardScaler()),
  ('svc', SVC(C=80, gamma=1, random_state=11))],
 'verbose': False,
 'standardscaler': StandardScaler(),
 'svc': SVC(C=80, gamma=1, random_state=11),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'svc__C': 80,
 'svc__break_ties': False,
 'svc__cache_size': 200,
 'svc__class_weight': None,
 'svc__coef0': 0.0,
 'svc__decision_function_shape': 'ovr',
 'svc__degree': 3,
 'svc__gamma': 1,
 'svc__kernel': 'rbf',
 'svc__max_iter': -1,
 'svc__probability': False,
 'svc__random_state': 11,
 'svc__shrinking': True,
 'svc__tol': 0.001,
 'svc__verbose': False}

### Модель №2

In [11]:
pipe_tree = make_pipeline(
      StandardScaler(),
      RandomForestClassifier(random_state=my_seed),
)


# настроим параметры SVM с помощью сеточного поиска

param_grid = {
    'randomforestclassifier__n_estimators': [60],
    'randomforestclassifier__max_depth': [20, 30],
    'randomforestclassifier__max_features': [5, 10],
    'randomforestclassifier__criterion': ['entropy'],
    'randomforestclassifier__min_samples_split': [3],
}

# разбиения для перекрёстной проверки
kfold = KFold(n_splits=10, random_state=my_seed, shuffle=True)
gs2 = GridSearchCV(estimator=pipe_tree, param_grid=param_grid, scoring='accuracy', refit=True, cv=kfold,
n_jobs=-1)
# таймер
tic = time.perf_counter()
# запускаем сеточный поиск
gs2 = gs2.fit(X_train, y_train)
# таймер
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

C:\Users\a_zlo\anaconda3\envs\notebook\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\a_zlo\anaconda3\envs\notebook\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\a_zlo\anaconda3\envs\notebook\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\a_zlo\anaconda3\envs\notebook\lib\site-packages\sklearn\ensemble\_forest.py", line 467, in fit
    f

Сеточный поиск занял 3.37 секунд


In [12]:
# точность лучшей модели
np.around(gs2.best_score_, 3)

0.827

In [13]:
gs2.best_estimator_.get_params()

{'memory': None,
 'steps': [('standardscaler', StandardScaler()),
  ('randomforestclassifier',
   RandomForestClassifier(criterion='entropy', max_depth=30, max_features=5,
                          min_samples_split=3, n_estimators=60, random_state=11))],
 'verbose': False,
 'standardscaler': StandardScaler(),
 'randomforestclassifier': RandomForestClassifier(criterion='entropy', max_depth=30, max_features=5,
                        min_samples_split=3, n_estimators=60, random_state=11),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'randomforestclassifier__bootstrap': True,
 'randomforestclassifier__ccp_alpha': 0.0,
 'randomforestclassifier__class_weight': None,
 'randomforestclassifier__criterion': 'entropy',
 'randomforestclassifier__max_depth': 30,
 'randomforestclassifier__max_features': 5,
 'randomforestclassifier__max_leaf_nodes': None,
 'randomforestclassifier__max_samples': None,
 'randomforestclassifier__min_impurity_de

Максимальную метрику Acc = 82,7% на обучающей выборке получилось выбить с помощью второй модели. Получить Acc больше этого значения, к сожалению, не получилось

## Прогноз на отложенные наблюдения по лучшей модели

In [14]:
score = list()
score_models = list()
score.append(np.around(gs.best_score_, 3))
score_models.append('model_1')

score.append(np.around(gs2.best_score_, 3))
score_models.append('model_2')

In [15]:
pd.DataFrame({'Модель' : score_models, 'Acc' : score})

,Модель,Acc
0,model_1,0.798
1,model_2,0.827


In [16]:
# прогноз с помощью лучшей модели (модель 2)
y_hat = gs2.best_estimator_.predict(X=DF_predict[['fixed_acidity','volatile_acidity','citric_acid','residual_sugar','chlorides','free_sulfur_dioxide','pH','sulphates','alcohol']])
# точность
# характеристики точности
print(classification_report(DF_predict.Y, y_hat))

              precision    recall  f1-score   support

           0       0.80      0.73      0.76       237
           1       0.88      0.92      0.89       498

    accuracy                           0.85       735
   macro avg       0.84      0.82      0.83       735
weighted avg       0.85      0.85      0.85       735



Точность модели на отложных наблюдениях оказалась больше (0.85), чем точности моделей, полученные до этого на других наблюдениях (0.798; 0.827)